<a href="https://colab.research.google.com/github/CRcr0/CUE6156F21/blob/main/FB_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas

In [2]:
!pip install schedule

In [3]:
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 7.6 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import schedule
import time

def job():
    print("I'm working...")

schedule.every().day.at("07:00").do(job) # set time to run the program

while True:
    schedule.run_pending() # do it
    time.sleep(1)

In [4]:
import yfinance as yf
df = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "FB",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "10d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

         # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  1 of 1 completed


In [5]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2021-11-19,342.200012,352.100006,339.899994,345.299988,26488500
2021-11-22,349.049988,353.829987,340.510010,341.010010,27116800
2021-11-23,338.929993,341.399994,333.500000,337.250000,17225000
2021-11-24,336.000000,341.779999,332.809998,341.059998,13566200
2021-11-26,335.799988,337.750000,331.899994,333.119995,14750700


In [6]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [7]:
def prepare_data(df,forecast_col,forecast_out,test_size):
    label = df[forecast_col].shift(-forecast_out) #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #processing the feature array
    X_lately = X[-forecast_out:] #creating the column i want to use later in the predicting method
    X = X[:-forecast_out] # X that will contain the training and testing
    label.dropna(inplace=True) #dropping na values
    y = np.array(label)  # assigning Y
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=0) #cross validation

    response = [X_train,X_test , Y_train, Y_test , X_lately]
    return response

In [8]:
forecast_col = 'High'
forecast_out = 1
test_size = 0.2

In [9]:
X_train, X_test, Y_train, Y_test , X_lately =prepare_data(df,forecast_col,forecast_out,test_size)
learner = LinearRegression() #initializing linear regression model

learner.fit(X_train,Y_train) #training the linear regression model

LinearRegression()

In [10]:
score=learner.score(X_test,Y_test)#testing the linear regression model
forecast= learner.predict(X_lately) #set that will contain the forecasted data
response={}#creting json object
#response['test_score']=score
response['forecast_set']=forecast

print(response)

{'forecast_set': array([313.95088615])}


In [11]:
a = forecast
a

array([313.95088615])

In [12]:
b = df.iloc[9]["High"]
b

313.75

In [13]:
#relative error
c = a-b
d = c/b
print(d)

[0.00064027]
